# Taxa de rendimento no ensino básico brasileiro
Este notebook será usado para fazer análises baseadas no dataframe disponibilizado pelo governo federal sobre o rendimentos dos alunos de diversas escolas espalhadas pelo país e que pode ser verificado através do link http://dados.gov.br/dataset/taxas-de-rendimento-escolar-na-educacao-basica?fbclid=IwAR1k-zIfTDd7K6rWWhIYY9JPyfkjYI8Pm83tY7_K29ZWntVCx5vMGoPCpMM 

O dataset usado é "Taxas - município 2010"
Serão apresentados os dados recolhidos, as hipóteses criadas, bem como se foram validadas ou não e algumas visualizações e inferências importantes. 

In [93]:
import pandas as pd
from unicodedata import normalize 

In [94]:
file = r'data/TX RENDIMENTO MUNICIPIOS 2010.xls'
column_labels = [
    'ano', 'regiao', 'uf', 'codigo_municipio', 'nome_municipio', 'localizacao', 'rede',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]
df_municipios = pd.read_excel(file, skiprows=8, nrows=65460, names=column_labels, na_values='--')

df_municipios.head()

,ano,regiao,uf,codigo_municipio,nome_municipio,localizacao,rede,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
0,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Estadual,100.0,35.1,70.6,...,NaN,3.4,6.0,4.2,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Municipal,97.9,88.3,88.6,...,11.0,1.2,10.3,6.1,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Publico,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Total,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Total,Estadual,98.2,83.3,82.0,...,8.2,2.0,9.2,6.1,18.3,12.1,8.4,NaN,NaN,14.1


In [95]:
normalize_string = lambda x: normalize('NFKD', x.upper()).encode('ASCII','ignore').decode("utf-8")

file = r'data/pib_municipios.xls'
df_pib_municipios = pd.read_excel(file, usecols=[0,5], skiprows=8, names=['local', 'pib'], nrows=5601).dropna()
df_pib_municipios['local'] = df_pib_municipios['local'].apply(normalize_string)

df_pib_municipios.head()

,local,pib
0,NORTE,2.015107e+08
1,RONDONIA,2.356064e+07
2,ALTA FLORESTA D'OESTE,3.356437e+05
3,ALTO ALEGRE DOS PARECIS,1.590252e+05
4,ALTO PARAISO,2.015904e+05


In [96]:
municipio_to_pib = {
    'AROEIRA DO ITAIM': 'AROEIRAS DO ITAIM',
    'BARRA D ALCANTARA': 'BARRA D\'ALCANTARA',
    'BATAIPORA': 'BATAYPORA',
    'BELA VISTA DO CAROBA': 'BELA VISTA DA CAROBA',
    'BELEM DE SAO FRANCISCO': 'BELEM DO SAO FRANCISCO',
    'CHIAPETA': 'CHIAPETTA',
    'COUTO DE MAGALHAES': 'COUTO MAGALHAES',
    'CUVERLANDIA': 'CURVELANDIA',
    'GOIO-ERE': 'GOIOERE',
    'GOVERNADOR LOMANTO JUNIOR': 'BARRO PRETO',
    'ITAMARACA': 'ILHA DE ITAMARACA',
    'LAGOA DO ITAENGA': 'LAGOA DE ITAENGA',
    'OLHO D AGUA DO PIAUI': 'OLHO D\'AGUA DO PIAUI',
    'OLHOS-D AGUA': 'OLHOS-D\'AGUA',
    'PARATI': 'PARATY',
    'PASSA VINTE': 'PASSA-VINTE',
    'PICARRAS': 'BALNEARIO PICARRAS',
    'PINDARE MIRIM': 'PINDARE-MIRIM',
    'PINGO D AGUA': 'PINGO-D\'AGUA',
    'SANTANA DO LIVRAMENTO': 'SANT\'ANA DO LIVRAMENTO',
    'SAO DOMINGOS DE POMBAL': 'POMBAL',
    'SAO FELIPE D OESTE': 'SAO FELIPE D\'OESTE',
    'SAO VALERIO DA NATIVIDADE': 'SAO VALERIO',
    'SAO VICENTE DO SERIDO': 'SERIDO',
    'SEM PEIXE': 'SEM-PEIXE',
    'TOCOS DO MOGI': 'TOCOS DO MOJI',
    'TRAJANO DE MORAIS': 'TRAJANO DE MORAES'
}

pib_to_municipio = {
    'MOJI MIRIM': 'MOGI MIRIM',
}

df_municipios.replace(municipio_to_pib, inplace=True)
df_pib_municipios.replace(pib_to_municipio, inplace=True)

df_municipios = df_municipios[df_municipios.nome_municipio != 'CAMPO DE SANTANA']

pass

In [97]:
df_municipios = pd.merge(df_municipios, df_pib_municipios, left_on='nome_municipio', right_on='local', how='left')
df_municipios.drop(['local'], axis=1, inplace=True)

cols = df_municipios.columns.tolist()
cols = cols[:6] + cols[-1:] + cols[7:-1]
df_municipios = df_municipios[cols]

df_municipios.head()

,ano,regiao,uf,codigo_municipio,nome_municipio,localizacao,pib,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
0,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,335643.702,100.0,35.1,70.6,...,NaN,3.4,6.0,4.2,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,335643.702,97.9,88.3,88.6,...,11.0,1.2,10.3,6.1,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,335643.702,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,335643.702,98.1,79.3,86.0,...,11.0,1.6,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Total,335643.702,98.2,83.3,82.0,...,8.2,2.0,9.2,6.1,18.3,12.1,8.4,NaN,NaN,14.1


In [119]:
mean_func = lambda col: col.mean()
sum_func = lambda col: col.sum()

agg_map = {
    'pib': sum_func,
}
agg_map.update({
    k: mean_func for k in df_municipios.columns.tolist()[7:]
})

df = df_municipios.dropna().groupby('uf').agg(agg_map)
df

,pib,aprovacao_fundamental_1,aprovacao_fundamental_2,aprovacao_fundamental_3,aprovacao_fundamental_4,aprovacao_fundamental_5,aprovacao_fundamental_6,aprovacao_fundamental_7,aprovacao_fundamental_8,aprovacao_fundamental_9,...,abandono_fundamental_9,abandono_fundamental_1:5,abandono_fundamental_6:9,abandono_fundamental_1:9,abandono_medio_1,abandono_medio_2,abandono_medio_3,abandono_medio_4,abandono_medio_0,abandono_medio_0:4
uf,,,,,,,,,,,,,,,,,,,,,
AL,7.268454e+07,90.100000,87.983333,73.683333,84.883333,82.666667,54.633333,63.500000,68.066667,72.000000,...,16.300000,5.616667,17.833333,12.600000,25.633333,19.500000,14.900000,9.800000,3.133333,20.733333
BA,7.734374e+07,95.500000,87.850000,72.375000,87.250000,82.525000,64.200000,70.375000,76.300000,76.100000,...,7.900000,3.700000,8.450000,6.000000,24.100000,19.725000,16.225000,13.150000,0.000000,20.425000
CE,7.839876e+06,95.050000,98.300000,82.275000,87.100000,88.275000,77.725000,82.525000,86.625000,90.000000,...,5.075000,2.400000,5.450000,3.825000,14.500000,11.100000,8.050000,0.000000,0.400000,11.300000
ES,9.987718e+07,99.600000,99.100000,89.500000,88.900000,91.650000,78.150000,83.150000,85.000000,89.350000,...,2.750000,0.800000,3.150000,1.900000,10.300000,7.650000,5.650000,0.000000,0.900000,7.800000
GO,5.317569e+07,98.300000,91.100000,92.175000,95.025000,96.275000,87.700000,84.525000,85.250000,85.975000,...,7.000000,0.950000,6.325000,3.625000,10.300000,7.950000,8.400000,1.650000,4.200000,9.050000
MA,1.245236e+08,96.259574,94.172340,90.289362,89.312766,90.178723,81.727660,83.634043,85.965957,86.093617,...,6.753191,1.863830,5.087234,3.444681,15.463830,14.448936,10.442553,5.910638,32.702128,13.887234
MG,1.230511e+07,98.625000,90.800000,88.100000,92.100000,94.700000,77.600000,82.950000,85.025000,80.300000,...,5.800000,0.800000,5.300000,2.850000,13.550000,12.225000,12.025000,18.150000,7.850000,13.075000
MT,9.649436e+07,99.373333,95.463333,93.460000,95.333333,96.076667,93.650000,97.710000,97.680000,93.400000,...,2.253333,0.353333,1.100000,0.723333,13.370000,9.860000,9.710000,0.220000,15.180000,11.506667
PA,3.618372e+07,91.800000,91.350000,77.400000,86.350000,85.600000,71.300000,73.500000,77.600000,74.200000,...,6.150000,4.750000,6.800000,5.650000,19.900000,18.700000,18.900000,7.000000,1.800000,19.150000
